In [7]:
# import os
# import glob
# import numpy as np
# from scipy.io import loadmat
# import seaborn as sns
# import pandas as pd
# import matplotlib.pyplot as plt
# import timecorr as tc
# import math
# import scipy.spatial.distance as sd
# from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce
# corrsdir = '/Users/lucyowen/Desktop/corrs_for_j'
# factors = 100

# if factors == 100:
#     pieman_name = '../../data/pieman_ica100.mat'
# else:
#     pieman_name = '../../data/pieman_data.mat'
    

# pieman_data = loadmat(pieman_name)  

# pieman_conds = ['word']
# data = []
# conds = []
# for c in pieman_conds:
#     if c == 'paragraph':
#         if factors == 700:
#             next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
#         else:
#             next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
#     else:
#         next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
#     data.extend(next_data)
#     conds.extend([c]*len(next_data))

# all_data = np.array(data)
# conds = np.array(conds)
# cfun = isfc
# rfun = 'eigenvector_centrality'
# width = 10
# wp = 'gaussian'
# cond = 'intact'
# level = 1

# gaussian = {'name': 'Gaussian', 'weights': tc.gaussian_weights, 'params': {'var': width}}

# weights_paramter = eval(wp)


# weights_fun=weights_paramter['weights']
# weights_params=weights_paramter['params']
# combine = corrmean_combine  
# group_chunks = 2


# ## just for word group 2
# #### RUN FOR 700 FACTORS ON CLUSTER
# for c in pieman_conds:
#     g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
#     for i in range(2):
#         data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=rfun,
#                                                      weights_function=weights_fun, weights_params=weights_params))
#         lev = np.asarray(tc.timecorr([x for x in data_1], cfun=isfc, rfun=None,
#                                                      weights_function=weights_fun, weights_params=weights_params))
#         corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
#         np.save(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy'), corr)




## Cluster code to calculate correlations for each order - up to 4th order

In [19]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce

factors = 700
if factors == 100:
    pieman_name = '../../data/pieman_ica100.mat'
else:
    pieman_name = '../../data/pieman_data.mat'



pieman_data = loadmat(pieman_name)  

pieman_conds = ['rest']

data = []
conds = []


for c in pieman_conds:
    if c == 'paragraph':
        if factors == 700:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 3)[0]))
        else:
            next_data = list(map(lambda i: pieman_data[c][:, i][0], np.where(np.arange(pieman_data[c].shape[1]) != 0)[0]))
    else:
        next_data = list(map(lambda i: pieman_data[c][:, i][0], np.arange(pieman_data[c].shape[1])))
    data.extend(next_data)
    conds.extend([c]*len(next_data))


all_data = np.array(data)
conds = np.array(conds)
cfun = isfc
rfun = 'eigenvector_centrality'
width = 50
wp = 'laplace'

results_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results/'

corrsdir = os.path.join(results_dir, wp + '_' + str(50) + '_corrs')

laplace = {'name': 'Laplace', 'weights': tc.laplace_weights, 'params': {'scale': width}}

weights_paramter = eval(wp)


weights_fun=weights_paramter['weights']
weights_params=weights_paramter['params']
combine = corrmean_combine  
group_chunks = 2


for c in pieman_conds:
    g_inds = np.array_split(list(range(np.shape(all_data[conds == c])[0])), group_chunks)
    for i in range(2):
        if os.path.exists(os.path.join(corrsdir, 'd_1_'+ c +'_'+str(i+1)+'.npy')):
            data_1 = np.load(os.path.join(corrsdir, 'd_1_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_1 = np.asarray(tc.timecorr([x for x in all_data[conds == c][g_inds[i]]], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data_1, axis=2)), axis=2))
            np.save(os.path.join(corrsdir, 'd_1_'+c+'_'+str(i+1)+'.npy'), data_1)
            np.save(os.path.join(corrsdir, 'lev_1_'+c+'_'+str(i+1)+'.npy'), corr)
        if os.path.exists(os.path.join(corrsdir, 'd_1_r_'+ c +'_'+str(i+1)+'.npy')):
            data_1_r = np.load(os.path.join(corrsdir, 'd_1_r_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_1_r = np.asarray(reduce([x for x in data_1], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_1_r_'+ c +'_'+str(i+1)+'.npy'), data_1_r)
        del data_1
        if os.path.exists(os.path.join(corrsdir, 'd_2_'+ c +'_'+str(i+1)+'.npy')):
            data_2 = np.load(os.path.join(corrsdir, 'd_2_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_2 = np.asarray(tc.timecorr([x for x in data_1_r], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data_2, axis=2)), axis=2))
            np.save(os.path.join(corrsdir, 'd_2_'+ c +'_'+str(i+1)+'.npy'), data_2)
            np.save(os.path.join(corrsdir, 'lev_2_'+c+'_'+str(i+1)+'.npy'), corr)
        if os.path.exists(os.path.join(corrsdir, 'd_2_r_'+ c +'_'+str(i+1)+'.npy')):
            data_2_r = np.load(os.path.join(corrsdir, 'd_2_r_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_2_r = np.asarray(reduce([x for x in data_2], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_2_r_'+ c +'_'+str(i+1)+'.npy'), data_2_r)
        del data_2
        if os.path.exists(os.path.join(corrsdir, 'd_3_'+ c +'_'+str(i+1)+'.npy')):
            data_3 = np.load(os.path.join(corrsdir, 'd_3_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_3 = np.asarray(tc.timecorr([x for x in data_2_r], cfun=isfc, rfun=None,
                                                         weights_function=weights_fun, weights_params=weights_params))
            corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(data_3, axis=2)), axis=2))
            np.save(os.path.join(corrsdir, 'd_3_'+ c +'_'+str(i+1)+'.npy'), data_3)
            np.save(os.path.join(corrsdir, 'lev_3_'+c+'_'+str(i+1)+'.npy'), corr)
        if os.path.exists(os.path.join(corrsdir, 'd_3_r_'+ c +'_'+str(i+1)+'.npy')):
            data_3_r = np.load(os.path.join(corrsdir, 'd_3_r_'+ c +'_'+str(i+1)+'.npy'))
        else:
            data_3_r = np.asarray(reduce([x for x in data_3], rfun=rfun))
            np.save(os.path.join(corrsdir, 'd_3_r_'+ c +'_'+str(i+1)+'.npy'), data_3_r)
        del data_3
        lev = np.asarray(tc.timecorr([x for x in data_3_r], cfun=isfc, rfun=None,
                                                     weights_function=weights_fun, weights_params=weights_params))
        corr = tc.helpers.z2r(np.mean(tc.helpers.r2z(np.stack(lev, axis=2)), axis=2))
        np.save(os.path.join(corrsdir, 'lev_4_'+c+'_'+str(i+1)+'.npy'), corr)
        del lev

IndexError: invalid index to scalar variable.

In [3]:
import os
import glob
import numpy as np
from scipy.io import loadmat
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import timecorr as tc
import math
import scipy.spatial.distance as sd
from timecorr.helpers import isfc, autofc, mean_combine, corrmean_combine, vec2mat, reduce


In [4]:
data_dir = '/dartfs/rc/lab/D/DBIC/CDL/f002s72/timecorr_paper/pieman/results'
corrs_dir = os.path.join(data_dir, 'laplace_50_corrs')

## For sliced correlations:

In [ ]:
levels = [1,2, 3, 4]
groups = [1, 2]
conditions = ['intact', 'paragraph', 'word', 'rest']

n = .1
percentages = np.arange(n, 1 + n, n)
for l in levels:
    for c in conditions:
        for g in groups:
            con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}'+ f'_{g}.npy')
            try:
                corrs = np.load(con)
                mat_corrs = tc.helpers.vec2mat(corrs)
                sliced_corrs = np.array([])
                next_corrdir = os.path.join(data_dir, 'sliced_corrs', f'level_{l}', f'group {g}')
                if not os.path.exists(next_corrdir):
                    os.makedirs(next_corrdir)
                for p in percentages:
                    s = int(p*np.shape(mat_corrs)[-1])
                    slice_corr = mat_corrs[:, :, s-1]
                    if sliced_corrs.shape[0] == 0:
                        sliced_corrs = slice_corr
                    else:
                        sliced_corrs = np.dstack((sliced_corrs, slice_corr))
                np.save(os.path.join(next_corrdir, f'{c}.npy'), sliced_corrs)
            except:
                print('issue loading: ' + con)
                pass

## For averaged correlations:

In [ ]:
levels = [1,2, 3, 4]
groups = [1, 2]
conditions = ['intact', 'paragraph', 'word', 'rest']


for l in levels:
    for c in conditions:
        for g in groups:
            con = os.path.join(corrs_dir, f'lev_{l}'+ f'_{c}'+ f'_{g}.npy')
            try:
                corrs = np.load(con)
                mat_corrs = tc.helpers.vec2mat(corrs)

                next_corrdir = os.path.join(data_dir, 'mean_corrs', f'level_{l}', f'group {g}')
                if not os.path.exists(next_corrdir):
                    os.makedirs(next_corrdir)

                mean_corrs = mat_corrs.mean(axis=2)

                np.save(os.path.join(next_corrdir, f'{c}.npy'), mean_corrs)
            
            except:
                print('issue loading: ' + con)
                pass